In [1]:
from importlib import reload

import ecc
import helper
import script
import tx

In [2]:
from ecc import PrivateKey
from helper import double_sha256, little_endian_to_int
from script import multisig_redeem_script, p2sh_script

secrets = (b'jimmy@programmingblockchain.com test1', b'jimmy@programmingblockchain.com test2')

private_keys = [PrivateKey(little_endian_to_int(double_sha256(s))) for s in secrets]
points = [p.point for p in private_keys]

redeem_script = multisig_redeem_script(2, points)
h160 = redeem_script.hash160()
script_pubkey = p2sh_script(h160)
print(script_pubkey.address(testnet=True))

2MxEZNps15dAnGX5XaVwZWgoDvjvsDE5XSx


In [3]:
from ecc import PrivateKey
from helper import double_sha256, little_endian_to_int
from script import multisig_redeem_script, p2sh_script, Script
from tx import Tx, TxIn, TxOut

secrets = (b'jimmy@programmingblockchain.com test1', b'jimmy@programmingblockchain.com test2')

private_keys = [PrivateKey(little_endian_to_int(double_sha256(s))) for s in secrets]
points = [p.point for p in private_keys]

redeem_script = multisig_redeem_script(2, points)
prev_tx = bytes.fromhex('9a54471e52b35be0fe2477b41e600d7171905f53a2a275c29c478f38b4f3d278')
prev_index = 0
fee = int(100000000*0.001)

tx_in = TxIn(prev_tx, prev_index, Script([]), 0xffffffff)
h160 = redeem_script.hash160()
script_pubkey = p2sh_script(h160)
amount = tx_in.value(testnet=True) - fee - 1000000
tx_out_1 = TxOut(amount, script_pubkey)
h160 = bytes.fromhex('7144796678ef9edc1a86745c2a01f8fb923ed656')
script_pubkey = p2sh_script(h160)
tx_out_2 = TxOut(1000000, script_pubkey)

t = Tx(1, [tx_in], [tx_out_1, tx_out_2], 0, testnet=True)
t.sign_input_p2sh_multisig(0, private_keys, redeem_script)
print(t.serialize().hex())

010000000178d2f3b4388f479cc275a2a2535f9071710d601eb47724fee05bb3521e47549a00000000da00483045022100aa47d20d5eb5b2d92c7b0a641194eb5a482b159147fe28c42f4c2f68ed961494022016b7258e9b74173c23b77d6dcb660d2b5bf7b588e70fbe54a3a65d52066ea30d01473044022040da6a008eff920f96382071cbcfd31f779e5291e4b8741ece8fcbe90750b6700220074f341b62a8e03c49be15c83ceabbe08dbdb3a2aa7b8b44de3141769e0e436d014752210223136797cb0d7596cb5bd476102fe3aface2a06338e1afabffacf8c3cab4883c210385c865e61e275ba6fda4a3167180fc5a6b607150ff18797ee44737cd0d34507b52aeffffffff02d7a57a000000000017a91436b865d5b9664193ea1db43d159edf9edf9438028740420f000000000017a9147144796678ef9edc1a86745c2a01f8fb923ed6568700000000


In [4]:
from ecc import PrivateKey
from helper import double_sha256, hash160, little_endian_to_int
from script import p2wpkh_script, p2sh_script, Script
from tx import Tx, TxIn, TxOut

secret = b'jimmy@programmingblockchain.com test1'

private_key = PrivateKey(little_endian_to_int(double_sha256(secret)))
point = private_key.point
real_h160 = hash160(point.sec())

redeem_script = p2wpkh_script(real_h160)
h160 = redeem_script.hash160()

print(p2sh_script(h160).address(testnet=True))

2N3a8NdfeA7SAurCGsd5k9AEYvbszipz3Jz


In [5]:
from ecc import PrivateKey
from helper import double_sha256, little_endian_to_int
from script import multisig_redeem_script, p2sh_script, Script
from tx import Tx, TxIn, TxOut

secret = b'jimmy@programmingblockchain.com test1'

private_key = PrivateKey(little_endian_to_int(double_sha256(secret)))
point = private_key.point
real_h160 = hash160(point.sec())

redeem_script = p2wpkh_script(real_h160)
h160 = redeem_script.hash160()

prev_tx = bytes.fromhex('770e48abf1051fe8df7d906d6a6589a3a6fe4da09edd5e4172ee04db3e88dfb8')
prev_index = 1
fee = int(100000000*0.001)

tx_in = TxIn(prev_tx, prev_index, Script([]), 0xffffffff)
script_pubkey = p2sh_script(h160)
amount = tx_in.value(testnet=True) - fee
tx_out = TxOut(amount, script_pubkey)

t = Tx(1, [tx_in], [tx_out], 0, testnet=True)
t.sign_input_p2sh_p2wpkh(0, private_key)
print(t.serialize_segwit().hex())

01000000000101b8df883edb04ee72415edd9ea04dfea6a389656a6d907ddfe81f05f1ab480e77010000001716001489eb4b9efba700895fd2b02a1b0d2bf8f0864935ffffffff01a0bb0d000000000017a9147144796678ef9edc1a86745c2a01f8fb923ed6568702483045022100a5bb908b7d66a8d85e461ac22263cf7d851dd316e1a45a76fcd7f8036f682e1b0220167b335118ba7235ca73bc60bf55bbd12005e1b7c99769e30a6f0e7eaad8031801210223136797cb0d7596cb5bd476102fe3aface2a06338e1afabffacf8c3cab4883c00000000
